In [1]:
%load_ext autoreload
%autoreload 2

from dask.distributed import Client

client = Client("tcp://127.0.0.1:40461")
client

Client Scheduler: tcp://127.0.0.1:40461 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 64 Memory: 720.00 GB


In [2]:
import dask.dataframe as dd

# df = dd.read_csv("../../data/train/viznet/all.csv")
# df

In [3]:
# import csv

# import nltk
# import regex as re

# df = df.assign(
#     data=df.data.str.replace("", "").replace("[A-Z]", "A", True)
#     .replace("[a-z]", "a", True)
#     .replace("[0-9]", "0", True)
# )

In [5]:
# df.to_csv(
#     f"../../data/train/viznet/pattern_all.csv",
#     quoting=csv.QUOTE_ALL,
#     single_file=True,
#     index=False,
# )

In [9]:
df = dd.read_csv("../../data/train/viznet/pattern_all.csv")
df

,data
npartitions=46,
,object
,...
...,...
,...
,...


In [12]:
import csv
import nltk
import numpy as np

df = df.persist()

def ngrams(x, n):
    x = str(x)
    try:
        return ["".join(ngram) for ngram in nltk.ngrams(list(x), n)]
    except Exception as e:
        print(e)
        return [x]


for i in range(2, 5):
    df[f"{i}_gram"] = df["data"].apply(ngrams, meta=(f"{i}_gram", np.array([""])), n=i)
    ngram_df = df.explode(f"{i}_gram")
    value2count = ngram_df[f"{i}_gram"].value_counts()
    value2count.to_frame().to_csv(
        f"../../data/train/viznet/{i}_gram_pattern_count.csv",
        single_file=True,
        quoting=csv.QUOTE_ALL,
    )
    df = df.drop(f"{i}_gram", axis=1)